# D213 - Advanced Data Analytics
### Task 1: Time Series Modeling 
___

## Part I: Research Question

### A1: Proposal of Question

### A2: Objectives and Goals

## Part II: Method Justification

### B1: Summary of Assumptions

## Part III: Data Preparation

### C1: Line Graph Visualization

### C2: Time Step Formatting

### C3: Stationarity

### C4: Steps to Prepare the Data

### C5: Prepared Dataset

## Part IV: Model Identification and Analysis

### D1: Report Findings and Visualizations

### D2: ARIMA Model

### D3: Forecasting Using ARIMA Model

### D4: Output and Calculations

### D5: Code

## Part V: Data Summary and Implications

### E1: Results

### E2: Annotated Visualization

### E3: Recommendations